In [1]:
import os
import utils.paths as plh
import utils.read_utils as hlpread
import utils.write_utils as hlwrite

In [2]:
PROJECT_ROOT = plh.get_project_root()
PROJECT_ROOT

WindowsPath('F:/github_workspace/employee_access_resource_classifier')

In [3]:
train_data = os.path.join(PROJECT_ROOT, 
                          hlpread.read_yaml_key('data_source.data_folders'),
                          hlpread.read_yaml_key('data_source.training_data_folder.folder'),
                          hlpread.read_yaml_key('data_source.training_data_folder.train'),
                         )

train_data

'F:\\github_workspace\\employee_access_resource_classifier\\data\\train.csv'

In [4]:
db_train = hlpread.read_csv(train_data)

print(db_train.shape)
db_train.head()

(32769, 10)


,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


#### Drop records of the RESOURCE with same product description but different MAG_ID ####

We shall drop +ve sample.

   - Consider the usecase if an unauthorised employee get access to the resource they are not support to access. It is riskey.
   - Beside if any individual employee id denied access to resource they can request the IT team for access.


In [5]:
#Records found while EDA with same description by difference in ACTION and MAG_ID
db_train.drop(db_train[db_train.RESOURCE.isin([27797, 27831, 36629, 77955, 81502])][db_train.ACTION ==1].index, axis = 0, inplace = True)
db_train.shape

C:\Users\smithau8\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(32764, 10)

In [6]:
#Drop the duplicate records from db_train
print(f'Before droping duplicate records: {db_train.shape[0]}')
db_train.drop_duplicates(subset = ['ACTION', 'RESOURCE', 
                                   'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2', 'ROLE_DEPTNAME', 
                                   'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY','ROLE_CODE'],
                         inplace = True                         
                         )

db_train.shape
print(f'After droping duplicate records: {db_train.shape[0]}')

Before droping duplicate records: 32764
After droping duplicate records: 30391


In [7]:
db_train.ACTION.value_counts()

1    28551
0     1840
Name: ACTION, dtype: int64

In [8]:
db_train['ACTION'].mean()

0.9394557599289264

In [9]:
db_train.columns

Index(['ACTION', 'RESOURCE', 'MGR_ID', 'ROLE_ROLLUP_1', 'ROLE_ROLLUP_2',
       'ROLE_DEPTNAME', 'ROLE_TITLE', 'ROLE_FAMILY_DESC', 'ROLE_FAMILY',
       'ROLE_CODE'],
      dtype='object')

In [10]:
#Save
save_path = os.path.join(PROJECT_ROOT, 
                         hlpread.read_yaml_key('data_source.data_folders'),
                         hlpread.read_yaml_key('data_source.prepared.folder'),
                         hlpread.read_yaml_key('data_source.prepared.clean_train'),
                        )
hlwrite.save_to_parquet(db_train, save_path)